In [1]:
from numba import njit
from hftbacktest import (
    SingleAssetHftBacktest_,
    HftBacktest,
    FeedLatency,
    SquareProbQueueModel,
    Linear,
    Stat,
    BUY,
    SELL,
    GTX,
    GTC, 
    ConstantLatency,
    PartialFillExchange
)
from hftbacktest import COL_EXCH_TIMESTAMP, COL_SIDE, COL_PRICE, COL_QTY
from hftbacktest import COL_EXCH_TIMESTAMP, COL_SIDE, COL_PRICE, COL_QTY
from numba.typed import List
from numba.types import Tuple, float64
import numpy as np

In [2]:
@njit
def simple_two_sided_quote(hbt: SingleAssetHftBacktest_, stat, money=10000):
    max_position = 10
    half_spread = hbt.tick_size
    skew = 0.01
    order_qty = 1
    last_order_id = -1
    order_id = 0
 
    bought = 1
    # Checks every 0.1s
    while hbt.elapse(100_000):
        # Clears cancelled, filled or expired orders.
        hbt.clear_inactive_orders()

        # Obtains the current mid-price and compute the reservation price.
        mid_price = (hbt.best_bid + hbt.best_ask) / 2.0

        # 
        
        buy_order_price = 133.5
        sell_order_price = 134.5
        last_order_id = -1
        # Cancel all outstanding orders
        for order in hbt.orders.values():
            if order.cancellable:
                hbt.cancel(order.order_id)
                last_order_id = order.order_id

        # All order requests are considered to be requested at the same time.
        # Waits until one of the order cancellation responses is received.
        if last_order_id >= 0:
            hbt.wait_order_response(last_order_id)

        # Clears cancelled, filled or expired orders.
        hbt.clear_inactive_orders()

        if hbt.position < max_position and (hbt.position * hbt.best_bid) < money:
            # Submits a new post-only limit bid order.
            order_id += 1
            hbt.submit_buy_order(
                order_id,
                buy_order_price,
                order_qty,
                GTC
            )
            last_order_id = order_id

        if hbt.position > -max_position and (hbt.position * hbt.best_ask) < money:
            # Submits a new post-only limit ask order.
            order_id += 1
            hbt.submit_sell_order(
                order_id,
                sell_order_price,
                order_qty,
                GTC
            )
            last_order_id = order_id

        # All order requests are considered to be requested at the same time.
        # Waits until one of the order responses is received.
        if last_order_id >= 0:
            hbt.wait_order_response(last_order_id)

        # Records the current state for stat calculation.
        stat.record(hbt)

@njit
def print_trades(hbt):
    while hbt.elapse(60 * 1e6):
        print('-------------------------------------------------------------------------------')
        print('current_timestamp:', hbt.current_timestamp)

        num = 0
        for trade in hbt.last_trades:
            if num > 10:
                print('...')
                break
            print(
                'exch_timestamp:',
                trade[COL_EXCH_TIMESTAMP],
                'buy' if trade[COL_SIDE] == 1 else 'sell',
                trade[COL_QTY],
                '@',
                trade[COL_PRICE]
            )
            num += 1

        hbt.clear_last_trades()
    return True


@njit
def print_3depth(hbt):
    while hbt.elapse(60 * 1e6):
        # a key of bid_depth or ask_depth is price in tick format.
        # (integer) price_tick = price / tick_size
        print('current_timestamp:', hbt.current_timestamp)
        i = 0
        for tick_price in range(hbt.best_ask_tick, hbt.high_ask_tick + 1):
            if tick_price in hbt.ask_depth:
                print(
                    'ask: ',
                    hbt.ask_depth[tick_price],
                    '@',
                    round(tick_price * hbt.tick_size, 3)
                )
                i += 1
                if i == 3:
                    break
        i = 0
        for tick_price in range(hbt.best_bid_tick, hbt.low_bid_tick - 1, -1):
            if tick_price in hbt.bid_depth:
                print(
                    'bid: ',
                    hbt.bid_depth[tick_price],
                    '@',
                    round(tick_price * hbt.tick_size, 3)
                )
                i += 1
                if i == 3:
                    break
    return True

@njit
def orderbookimbalance(hbt, out):
    while hbt.elapse(10 * 1e6):
        mid_price = (hbt.best_bid + hbt.best_ask) / 2.0

        sum_ask_qty_50bp = 0.0
        sum_ask_qty = 0.0
        for tick_price in range(hbt.best_ask_tick, hbt.high_ask_tick + 1):
            if tick_price in hbt.ask_depth:
                ask_price = tick_price * hbt.tick_size
                depth_from_mid = (ask_price - mid_price) / mid_price
                if depth_from_mid > 0.01:
                    break
                sum_ask_qty += hbt.ask_depth[tick_price]

                if depth_from_mid <= 0.005:
                    sum_ask_qty_50bp = sum_ask_qty


        sum_bid_qty_50bp = 0.0
        sum_bid_qty = 0.0
        for tick_price in range(hbt.best_bid_tick, hbt.low_bid_tick - 1, -1):
            if tick_price in hbt.bid_depth:
                bid_price = tick_price * hbt.tick_size
                depth_from_mid = (mid_price - bid_price) / mid_price
                if depth_from_mid > 0.01:
                    break
                sum_bid_qty += hbt.bid_depth[tick_price]

                if depth_from_mid <= 0.005:
                    sum_bid_qty_50bp = sum_bid_qty

        imbalance_50bp = sum_bid_qty_50bp - sum_ask_qty_50bp
        imbalance_1pct = sum_bid_qty - sum_ask_qty
        imbalance_tob = hbt.bid_depth[hbt.best_bid_tick] - hbt.ask_depth[hbt.best_ask_tick]

        out.append((hbt.current_timestamp, imbalance_tob, imbalance_50bp, imbalance_1pct))
    return True


In [3]:
# This backtest assumes market maker rebates.
# https://www.binance.com/en/support/announcement/binance-upgrades-usd%E2%93%A2-margined-futures-liquidity-provider-program-2023-04-04-01007356e6514df3811b0c80ab8c83bf
    
hbt = HftBacktest(
    [
        "/home/danny/hftbacktest/mbp/GOOG_20231215.npz"
    ],
    tick_size=0.01,
    lot_size=1,
    maker_fee=0,
    taker_fee=0.0,
    order_latency=ConstantLatency(0.0001, 0.0001),
    queue_model=SquareProbQueueModel(),
    asset_type=Linear,
    exchange_model=PartialFillExchange,
)
# print_3depth(hbt)
# stat = Stat(hbt)
print_trades(hbt)
# stat.summary(trading_days=1)

Load /home/danny/hftbacktest/mbp/GOOG_20231215.npz
loaded
-------------------------------------------------------------------------------
current_timestamp: 1702646469415000
-------------------------------------------------------------------------------
current_timestamp: 1702646529415000
-------------------------------------------------------------------------------
current_timestamp: 1702646589415000
-------------------------------------------------------------------------------
current_timestamp: 1702646649415000
-------------------------------------------------------------------------------
current_timestamp: 1702646709415000
-------------------------------------------------------------------------------
current_timestamp: 1702646769415000
-------------------------------------------------------------------------------
current_timestamp: 1702646829415000
-------------------------------------------------------------------------------
current_timestamp: 1702646889415000
--------------

True